# Surfalex formability predictions - MatFlow workflow analysis

This Jupyter notebook demonstrates the MatFlow workflows that were generated to investigate the formability of the Surfalex HF Al alloy. Running the cells in this notebook demonstrates how the MatFlow Python API can be used to inspect the results from, and perform further analysis on, MatFlow workflows.

This notebook and the five associated MatFlow workflows can be considered supplementary data to the following manuscript:

***'A novel integrated framework for reproducible formability predictions using virtual materials testing'***, A. J. Plowman, P. Jedrasiak, T. Jailin, P. Crowther, S. Mishra, P. Shanthraj, J. Quinta da Fonseca, in preparation.

In the work described in the above manuscript, we split the workflow into five sub-workflows, as follows:

1. Generate a representative volume element (with DAMASK and MTEX)
2. Fit single-crystal parameters for calibrated crystal plasticity (CP) simulations (with DAMASK)
3. Fit yield functions from full field CP simulations (with DAMASK and formable)
4. Estimate hardening curves from full field CP simulations (with DAMASK)
5. Perform Marciniak-Kukcyski simulations using the finite-element method (with Abaqus)

In [ ]:
%load_ext autoreload
%autoreload 2

## Import packages

In [ ]:
import pathlib

import numpy as np
import pandas as pd
from matflow import load_workflow
from formable.load_response import LoadResponse, LoadResponseSet
from formable.levenberg_marquardt import LMFitter
from formable.tensile_test import TensileTest

import utilities

## Utility functions

In [ ]:
FIG_EXPORT_DIR = pathlib.Path('../results/')
PLASTIC_TABLES_DIR = pathlib.Path('../results/simulated_plastic_tables')
PLASTIC_TABLES_DIR.mkdir(exist_ok=True, parents=True)

SHEET_DIRS = {'x': 'RD', 'y': 'TD', 'z': 'ND'}

## Download workflow HDF5 files from Zenodo

In [ ]:
all_workflow_paths = []
for wk_name, wk_info in utilities.read_data_yaml('../data/zenodo_URLs.yaml')['modelling_workflows'].items():
    wk_path_i = utilities.get_file_from_url('../data/modelling_workflows', name=wk_name + '.hdf5', **wk_info)    
    all_workflow_paths.append(wk_path_i)

(
    wkflow_1,
    wkflow_2,
    wkflow_3,
    wkflow_4,
    wkflow_5,
) = (
    load_workflow(i, full_path=True) for i in all_workflow_paths
)
    

## Workflow 1: Generate volume element

In [ ]:
print(wkflow_1)

## Workflow 2: Fit single-crystal parameters

In [ ]:
print(wkflow_2)

#### Show the experimental stress-strain data

In [ ]:
exp_tensile_test_dict = wkflow_2.tasks.get_tensile_test.elements[0].outputs.tensile_test
exp_tensile_test = TensileTest(**exp_tensile_test_dict)
exp_tensile_test.show(stress_strain_type='true')

#### Show the convergence of the stress-strain curve with iterations

Different elements in the `optimise_single_crystal_parameters` task correspond to different optimisation iterations. We would like to retrieve the element corresponding to the final iteration:

In [ ]:
final_iteration_element = wkflow_2.tasks.optimise_single_crystal_parameters.get_elements_from_iteration(-1)[0]

We can then reconstitue an `LMFitter` object from this element data, enabling a visualisation of the fitting process:

In [ ]:
lm_fitter_dict = final_iteration_element.outputs.levenberg_marquardt_fitter
lm_fitter = LMFitter.from_dict(lm_fitter_dict)
lm_fitter_fig = lm_fitter.show()
lm_fitter_fig

#### Produce a static plot for the manuscript

In [ ]:
lm_fitter_fig_static = utilities.plot_static_figure_single_crystal_fitting(lm_fitter)
lm_fitter_fig_static.write_image(str(FIG_EXPORT_DIR.joinpath('singleCrystalFitting.svg')))
lm_fitter_fig_static.show(config={'displayModeBar': False})

(Then use inkscape to generate a "compilable" figure for inclusion in the manuscript with: `inkscape -D --export-latex --export-type="pdf" singleCrystalFitting.svg`)

#### Initial trial parameters

In [ ]:
initial_parameters = wkflow_2.tasks.simulate_volume_element_loading.elements[0].inputs.single_crystal_parameters
utilities.pretty_print_single_crystal_parameters(initial_parameters)

#### Final optimised parameters

Take the final parameters for which a set of simulations were run. This is the second-to-last iteration, because the last iteration generates new parameters that would be used for simulations in the next iteration.

In [ ]:
final_parameters = wkflow_2.tasks.optimise_single_crystal_parameters.get_elements_from_iteration(-2)[0].outputs.single_crystal_parameters
utilities.pretty_print_single_crystal_parameters(final_parameters)

## Workflow 3: Fit yield functions

In [ ]:
print(wkflow_3)

In [ ]:
all_load_responses = [
    LoadResponseSet.from_dict(i.outputs.fitted_yield_functions)
    for i in wkflow_3.tasks.fit_yield_function.elements
]

#### Tables of fitted yield function parameters at all yield points

In [ ]:
all_fitted_params = utilities.show_all_fitted_yield_function_parameters(all_load_responses)

#### Example of accessing the parameters from a given yield point fit

In [ ]:
all_fitted_params['Barlat_Yld2004_18p'][10]

#### Yield function evolution - animation

In [ ]:
utilities.animate_yield_function_evolution(all_load_responses[1:], plane=[0,0,1], normalise=True, sheet_dirs=SHEET_DIRS)

#### Produce a static plot for the manuscript

In [ ]:
yield_funcs_fig_static = utilities.plot_static_figure_yield_function_type_comparison(all_load_responses[1:], yield_point=0.00275)
yield_funcs_fig_static.write_image(str(FIG_EXPORT_DIR.joinpath('yieldFuncComparison.svg')))
yield_funcs_fig_static.show(config={'displayModeBar': False})

(Then use inkscape to generate a "compilable" figure for inclusion in the manuscript with: `inkscape -D --export-latex --export-type="pdf" yieldFuncComparison.svg`)

#### Generate a parameter table for manuscript

In [ ]:
# Index 10 is the 0.00275 yield point
data = np.concatenate([
    utilities.get_latex_yield_func_params('Barlat_Yld2004_18p', all_fitted_params['Barlat_Yld2004_18p'][10], pad_to=18),
    utilities.get_latex_yield_func_params('Barlat_Yld91', all_fitted_params['Barlat_Yld91'][10], pad_to=18),
    utilities.get_latex_yield_func_params('Hill1948', all_fitted_params['Hill1948'][10], pad_to=18),        
]).T
print(pd.DataFrame(data=data).to_latex(header=False, index=False, escape=False))

#### Yield function evolution at selected yield points

##### Von Mises

In [ ]:
utilities.compare_yield_function_yield_points(all_load_responses[0], slice(0, -1, 10), plane=[0, 0, 1], sheet_dirs=SHEET_DIRS)

##### Hill 1948

In [ ]:
utilities.compare_yield_function_yield_points(all_load_responses[1], slice(0, -1, 10), plane=[0, 0, 1], sheet_dirs=SHEET_DIRS)

##### Barlat Yld91

In [ ]:
utilities.compare_yield_function_yield_points(all_load_responses[2], slice(0, -1, 10), plane=[0, 0, 1], sheet_dirs=SHEET_DIRS)

##### Barlat Yld2004-18p

In [ ]:
utilities.compare_yield_function_yield_points(all_load_responses[3], slice(0, -1, 10), plane=[0, 0, 1], sheet_dirs=SHEET_DIRS)

## Workflow 4: Estimate hardening curves

In [ ]:
print(wkflow_4)

#### Plot plastic stress-strain curves and hardening rates for each strain path

#### Produce a static plot for the manuscript

#### Interpolate values of stress for each strain path

In [ ]:
hardening_curve_tasks = {task.context: task for task in wkflow_4.tasks if task.name == 'simulate_volume_element_loading'}
hardening_data = utilities.collect_hardening_data(hardening_curve_tasks, yield_stress=95e6)
interpolated_data = utilities.show_interpolated_plastic_stress_strain_data(hardening_data)

#### Write out plastic stress-strain data for Abaqus workflow (Workflow 5)

In [ ]:
for strain_path in hardening_data.keys():
    path_i = PLASTIC_TABLES_DIR.joinpath(f'{strain_path}.csv')
    with open(path_i, 'w') as file:
        file.write('% Plastic table ({} strain path)\n% True stress (MPa), True strain\n'.format(strain_path))
        ordered_cols_df = interpolated_data.get((strain_path)).reindex(columns=interpolated_data.get((strain_path)).columns[::-1])    
        ordered_cols_df['interpolated_stress'] /= 1e6
        ordered_cols_df.to_csv(file, header=False, index=False, line_terminator='\n', float_format='%.5f')

## Workflow 5: Simulate Marciniak-Kuzynski analysis

In [ ]:
FLC = wkflow_5.tasks.find_forming_limit_curve.elements[0].outputs.forming_limit_curve

In [ ]:
utilities.show_FLC(FLC)